# Assignment 2

**Credits**: Federico Ruggeri, Eleonora Mancini, Paolo Torroni

**Keywords**: Human Value Detection, Multi-label classification, Transformers, BERT


# Contact

For any doubt, question, issue or help, you can always contact us at the following email addresses:

Teaching Assistants:

* Federico Ruggeri -> federico.ruggeri6@unibo.it
* Eleonora Mancini -> e.mancini@unibo.it

Professor:

* Paolo Torroni -> p.torroni@unibo.it

# Introduction

You are tasked to address the [Human Value Detection challenge](https://aclanthology.org/2022.acl-long.306/).

## Problem definition

Arguments are paired with their conveyed human values.

Arguments are in the form of **premise** $\rightarrow$ **conclusion**.

### Example:

**Premise**: *``fast food should be banned because it is really bad for your health and is costly''*

**Conclusion**: *``We should ban fast food''*

**Stance**: *in favour of*

<center>
    <img src="images/human_values.png" alt="human values" />
</center>

# [Task 1 - 0.5 points] Corpus

Check the official page of the challenge [here](https://touche.webis.de/semeval23/touche23-web/).

The challenge offers several corpora for evaluation and testing.

You are going to work with the standard training, validation, and test splits.

#### Arguments
* arguments-training.tsv
* arguments-validation.tsv
* arguments-test.tsv

#### Human values
* labels-training.tsv
* labels-validation.tsv
* labels-test.tsv

### Example

#### arguments-*.tsv
```

Argument ID    A01005

Conclusion     We should ban fast food

Stance         in favor of

Premise        fast food should be banned because it is really bad for your health and is costly.
```

#### labels-*.tsv

```
Argument ID                A01005

Self-direction: thought    0
Self-direction: action     0
...
Universalism: objectivity: 0
```

### Splits

The standard splits contain

   * **Train**: 5393 arguments
   * **Validation**: 1896 arguments
   * **Test**: 1576 arguments

### Annotations

In this assignment, you are tasked to address a multi-label classification problem.

You are going to consider **level 3** categories:

* Openness to change
* Self-enhancement
* Conversation
* Self-transcendence

**How to do that?**

You have to merge (**logical OR**) annotations of level 2 categories belonging to the same level 3 category.

**Pay attention to shared level 2 categories** (e.g., Hedonism). $\rightarrow$ [see Table 1 in the original paper.](https://aclanthology.org/2022.acl-long.306/)

#### Example

```
Self-direction: thought:    0
Self-direction: action:     1
Stimulation:                0
Hedonism:                   1

Openess to change           1
```

### Instructions

* **Download** the specificed training, validation, and test files.
* **Encode** split files into a pandas.DataFrame object.
* For each split, **merge** the arguments and labels dataframes into a single dataframe.
* **Merge** level 2 annotations to level 3 categories.

In [ ]:
!mkdir data
!curl https://zenodo.org/records/8248658/files/arguments-training.tsv?download=1 -o data/arguments-training.tsv
!curl https://zenodo.org/records/8248658/files/arguments-validation.tsv?download=1 -o data/arguments-validation.tsv
!curl https://zenodo.org/records/8248658/files/arguments-test.tsv?download=1 -o data/arguments-test.tsv
!curl https://zenodo.org/records/8248658/files/labels-training.tsv?download=1 -o data/labels-training.tsv
!curl https://zenodo.org/records/8248658/files/labels-validation.tsv?download=1 -o data/labels-validation.tsv
!curl https://zenodo.org/records/8248658/files/labels-test.tsv?download=1 -o data/labels-test.tsv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  988k  100  988k    0     0   485k      0  0:00:02  0:00:02 --:--:--  485k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  354k  100  354k    0     0   214k      0  0:00:01  0:00:01 --:--:--  214k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  283k  100  283k    0     0   191k      0  0:00:01  0:00:01 --:--:--  191k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  247k  100  247k    0     0   168k      0  0:00:01  0:00:01 --:--:--  168k
  % Total    % Received % Xferd  Average Speed   Tim

In [ ]:
import pandas as pd

arguments_train = pd.read_csv('data/arguments-training.tsv', sep='\t')
arguments_val = pd.read_csv('data/arguments-validation.tsv', sep='\t')
arguments_test = pd.read_csv('data/arguments-test.tsv', sep='\t')
labels_train = pd.read_csv('data/labels-training.tsv', sep='\t')
labels_val = pd.read_csv('data/labels-validation.tsv', sep='\t')
labels_test = pd.read_csv('data/labels-test.tsv', sep='\t')

In [ ]:
arguments_train.head()

,Argument ID,Conclusion,Stance,Premise
0,A01002,We should ban human cloning,in favor of,we should ban human cloning as it will only ca...
1,A01005,We should ban fast food,in favor of,fast food should be banned because it is reall...
2,A01006,We should end the use of economic sanctions,against,sometimes economic sanctions are the only thin...
3,A01007,We should abolish capital punishment,against,capital punishment is sometimes the only optio...
4,A01008,We should ban factory farming,against,factory farming allows for the production of c...


In [ ]:
labels_train.head()

,Argument ID,Self-direction: thought,Self-direction: action,Stimulation,Hedonism,Achievement,Power: dominance,Power: resources,Face,Security: personal,...,Tradition,Conformity: rules,Conformity: interpersonal,Humility,Benevolence: caring,Benevolence: dependability,Universalism: concern,Universalism: nature,Universalism: tolerance,Universalism: objectivity
0,A01002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,A01005,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,A01006,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,A01007,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
4,A01008,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,1,0,0,0


In [ ]:
print(labels_train.columns)

Index(['Argument ID', 'Self-direction: thought', 'Self-direction: action',
       'Stimulation', 'Hedonism', 'Achievement', 'Power: dominance',
       'Power: resources', 'Face', 'Security: personal', 'Security: societal',
       'Tradition', 'Conformity: rules', 'Conformity: interpersonal',
       'Humility', 'Benevolence: caring', 'Benevolence: dependability',
       'Universalism: concern', 'Universalism: nature',
       'Universalism: tolerance', 'Universalism: objectivity'],
      dtype='object')


In [ ]:
OPEN = 'Openness to change'
ENHANCE = 'Self-enhancement'
CONSERVE = 'Conservation'
TRANSCEND = 'Self-transcendence'

TOUCHE_VALUES = [OPEN, ENHANCE, CONSERVE, TRANSCEND]

value_mapping = {
    'Self-direction: thought' : [OPEN],
    'Self-direction: action' : [OPEN],
    'Stimulation' : [OPEN],
    'Hedonism' : [OPEN, ENHANCE],
    'Achievement' : [ENHANCE],
    'Power: dominance' : [ENHANCE],
    'Power: resources' : [ENHANCE],
    'Face' : [CONSERVE, ENHANCE],
    'Security: personal' : [CONSERVE],
    'Security: societal' : [CONSERVE],
    'Tradition' : [CONSERVE],
    'Conformity: rules' : [CONSERVE],
    'Conformity: interpersonal' : [CONSERVE],
    'Humility' : [TRANSCEND, CONSERVE],
    'Benevolence: caring' : [TRANSCEND],
    'Benevolence: dependability' : [TRANSCEND],
    'Universalism: concern' : [TRANSCEND],
    'Universalism: nature' : [TRANSCEND],
    'Universalism: tolerance' : [TRANSCEND],
    'Universalism: objectivity' : [TRANSCEND]
}

In [ ]:
def merge_values(element):
    new_element = { k : 0 for k in TOUCHE_VALUES }
    for key, value in element.items():
        raise RuntimeError
        if value == 1:
            for level_3_value in value_mapping[key]:
                new_element[level_3_value] = 1

    return new_element

def build_dataset(arguments, labels):
    new_dataset = []
    for argument_element, label_element in zip(arguments.iloc, labels.iloc):
        assert argument_element['Argument ID'] == label_element['Argument ID']
        new_element = { k : 0 for k in TOUCHE_VALUES }
        for level_2_value, level_3_values in value_mapping.items():
            if label_element[level_2_value] == 1:
                for level_3_value in level_3_values:
                    new_element[level_3_value] = 1

        for k in ['Premise', 'Stance', 'Conclusion']:
            new_element[k] = argument_element[k]
        new_dataset.append(new_element)

    return pd.DataFrame(new_dataset)

train_df = build_dataset(arguments_train, labels_train)
val_df = build_dataset(arguments_val, labels_val)
test_df = build_dataset(arguments_test, labels_test)

In [ ]:
train_df.head()

,Openness to change,Self-enhancement,Conservation,Self-transcendence,Premise,Stance,Conclusion
0,0,0,1,0,we should ban human cloning as it will only ca...,in favor of,We should ban human cloning
1,0,0,1,0,fast food should be banned because it is reall...,in favor of,We should ban fast food
2,0,1,1,0,sometimes economic sanctions are the only thin...,against,We should end the use of economic sanctions
3,0,0,1,1,capital punishment is sometimes the only optio...,against,We should abolish capital punishment
4,0,0,1,1,factory farming allows for the production of c...,against,We should ban factory farming


# [Task 2 - 2.0 points] Model definition

You are tasked to define several neural models for multi-label classification.

<center>
    <img src="images/model_schema.png" alt="model_schema" />
</center>

### Instructions

* **Baseline**: implement a random uniform classifier (an individual classifier per category).
* **Baseline**: implement a majority classifier (an individual classifier per category).

<br/>

* **BERT w/ C**: define a BERT-based classifier that receives an argument **conclusion** as input.
* **BERT w/ CP**: add argument **premise** as an additional input.
* **BERT w/ CPS**: add argument premise-to-conclusion **stance** as an additional input.

### Notes

**Do not mix models**. Each model has its own instructions.

You are **free** to select the BERT-based model card from huggingface.

#### Examples

```
bert-base-uncased
prajjwal1/bert-tiny
distilbert-base-uncased
roberta-base
```

### BERT w/ C

<center>
    <img src="images/bert_c.png" alt="BERT w/ C" />
</center>

### BERT w/ CP

<center>
    <img src="images/bert_cp.png" alt="BERT w/ CP" />
</center>

### BERT w/ CPS

<center>
    <img src="images/bert_cps.png" alt="BERT w/ CPS" />
</center>

### Input concatenation

<center>
    <img src="images/input_merging.png" alt="Input merging" />
</center>

### Notes

The **stance** input has to be encoded into a numerical format.

You **should** use the same model instance to encode **premise** and **conclusion** inputs.

In [ ]:
device = 'cuda'

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class RandomUniformClassifier(nn.Module):
    def __init__(self, output_shape, device='cuda'):
        super().__init__()
        self.output_shape = output_shape
        self.device = device

    def forward(self, x):
        return torch.rand((len(x),) + self.output_shape, device=self.device)

In [ ]:
class FixedOutputClassifier(nn.Module):
    def __init__(self, output, device):
        super().__init__()
        self.output = output.to(device)

    def forward(self, x):
        return self.output.unsqueeze(dim=0).repeat(len(x), 1)

def build_majority_classifier(dataloader, device):
    count = 0
    total_length = 0

    for _, _, _, labels in dataloader:
        count += labels.sum(axis=0)
        total_length += len(labels)

    most_common_output = (count >= total_length / 2).long()

    return FixedOutputClassifier(most_common_output, device)

In [ ]:
class ClassificationHead(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.sigmoid(x)

        return x

In [ ]:
class BERTC(nn.Module):
    def __init__(self, tokenizer, base_model, base_model_size, hidden_size, num_classes=4, eos_id=2, device='cuda'):
        super().__init__()
        self.tokenizer = tokenizer
        self.base_model = base_model
        heads = []
        for _ in range(num_classes):
            heads.append(ClassificationHead(base_model_size, hidden_size).to(device))

        self.heads = nn.ModuleList(heads)
        self.eos_id = eos_id
        self.device = device

    def forward(self, premise, conclusion, stance):
        with torch.no_grad():
            encoded_inputs = self.tokenizer(conclusion, return_tensors='pt', padding=True).to(self.device)
            # model_output is [B, L, D]
            model_output = self.base_model(**encoded_inputs)['pooler_output']

        outputs = []
        for head in self.heads:
            outputs.append(head(model_output).squeeze(dim=-1))

        return torch.stack(outputs, dim=1)

In [ ]:
class BERTCP(BERTC):
    def __init__(self, tokenizer, base_model, base_model_size, hidden_size, num_classes=4, eos_id=2, device='cuda'):
        super().__init__(tokenizer, base_model, base_model_size, hidden_size, num_classes=num_classes, eos_id=eos_id, device=device)

    def forward(self, premise, conclusion, stance):
        with torch.no_grad():
            encoded_inputs_premise = self.tokenizer(premise, return_tensors='pt', padding=True).to(self.device)
            encoded_inputs_conclusion = self.tokenizer(conclusion, return_tensors='pt', padding=True).to(self.device)

            # size is [B, D] for both of them
            output_premise = self.base_model(**encoded_inputs_premise)['pooler_output']
            output_conclusion = self.base_model(**encoded_inputs_conclusion)['pooler_output']

            concatenated_output = torch.cat([output_premise, output_conclusion], dim=-1)

        outputs = []
        for head in self.heads:
            outputs.append(head(concatenated_output).squeeze(dim=-1))

        return torch.stack(outputs, dim=1)

In [ ]:
class BERTCPS(BERTC):
    def __init__(self, tokenizer, base_model, base_model_size, hidden_size, num_classes=4, eos_id=2, device='cuda'):
        super().__init__(tokenizer, base_model, base_model_size, hidden_size, num_classes=num_classes, eos_id=eos_id, device=device)

    def forward(self, premise, conclusion, stance):
        with torch.no_grad():
            stance = stance.to(self.device)
            encoded_inputs_premise = self.tokenizer(premise, return_tensors='pt', padding=True).to(self.device)
            encoded_inputs_conclusion = self.tokenizer(conclusion, return_tensors='pt', padding=True).to(self.device)

            # size is [B, D] for both of them
            output_premise = self.base_model(**encoded_inputs_premise)['pooler_output']
            output_conclusion = self.base_model(**encoded_inputs_conclusion)['pooler_output']

            # stance has size [B, 2]
            concatenated_output = torch.cat([output_premise, output_conclusion, stance], dim=-1)

        outputs = []
        for head in self.heads:
            outputs.append(head(concatenated_output).squeeze(dim=-1))

        return torch.stack(outputs, dim=1)

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
base_model = BertModel.from_pretrained("bert-base-uncased").to(device)

for param in base_model.parameters():
    param.requires_grad = False

In [ ]:
class ValueDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.premises = list(df['Premise'].values)
        self.stances = torch.Tensor([
            [1, 0] if x['Stance'].lower() == 'in favor of' else [0, 1]
            for x in df.iloc
        ])
        self.conclusions = list(df['Conclusion'].values)
        self.labels = torch.stack([
            torch.Tensor(df[k].values)
            for k in TOUCHE_VALUES
        ], axis=-1)

    def __len__(self):
        return len(self.stances)

    def __getitem__(self, idx):
        return (self.premises[idx], self.conclusions[idx], self.stances[idx], self.labels[idx])

In [ ]:
train_dataset = ValueDataset(train_df)
val_dataset = ValueDataset(val_df)
test_dataset = ValueDataset(test_df)

In [ ]:
train_dataset[0]

('we should ban human cloning as it will only cause huge issues when you have a bunch of the same humans running around all acting the same.',
 'We should ban human cloning',
 tensor([1., 0.]),
 tensor([0., 0., 1., 0.]))

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
random_model = RandomUniformClassifier((4,), device=device)
majority_model = build_majority_classifier(train_dataloader, device=device)

bertc_model = BERTC(tokenizer, base_model, 768, 10, device=device)
bertcp_model = BERTCP(tokenizer, base_model, 768 * 2, 10, device=device)
bertcps_model = BERTCPS(tokenizer, base_model, 768 * 2 + 2, 10, device=device)

In [ ]:
from tqdm import tqdm

from torch.optim import Adam

bce_loss = nn.BCELoss(reduction='sum')

class MulticlassBCE:
    def __init__(self, num_labels):
        self.num_labels = num_labels

    def __call__(self, outputs, labels):
        loss = 0

        for i in range(self.num_labels):
            loss += bce_loss(outputs[:, i], labels[:, i])

        return loss / self.num_labels

loss_function = MulticlassBCE(num_labels=4)

def train(model, train_loader, val_loader, loss_function, epochs, lr):
    optimizer = Adam(model.parameters(), lr=lr)

    for i in range(epochs):
        model.train()
        train_loss = 0
        for premises, conclusions, stances, labels in tqdm(train_loader, desc=f'Epoch {i + 1}/{epochs}'):
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(premises, conclusions, stances)
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.cpu().item()
        print(train_loss / len(train_loader))

        model.eval()
        with torch.no_grad():
            total_val_loss = 0
            for premises, conclusions, stances, labels in val_loader:
                labels = labels.to(device)
                outputs = model(premises, conclusions, stances)
                loss = loss_function(outputs, labels)
                total_val_loss += loss.cpu().item()
        print(total_val_loss / len(val_loader))

In [ ]:
train(bertc_model, train_dataloader, val_dataloader, loss_function, 20, 0.001)

Epoch 1/20: 100%|██████████| 169/169 [00:08<00:00, 20.87it/s]


19.652498719254893
19.045478558540346


Epoch 2/20: 100%|██████████| 169/169 [00:07<00:00, 22.22it/s]


19.51554853676339
19.172083791097005


Epoch 3/20: 100%|██████████| 169/169 [00:07<00:00, 21.77it/s]


...

19.068274797067133
18.935191575686137


Epoch 19/20: 100%|██████████| 169/169 [00:07<00:00, 22.52it/s]


19.049959148880998
19.157809654871624


Epoch 20/20: 100%|██████████| 169/169 [00:07<00:00, 21.50it/s]


19.06236620344354
19.217957695325214


# [Task 3 - 0.5 points] Metrics

Before training the models, you are tasked to define the evaluation metrics for comparison.

### Instructions

* Evaluate your models using per-category binary F1-score.
* Compute the average binary F1-score over all categories (macro F1-score).

### Example

You start with individual predictions ($\rightarrow$ samples).

```
Openess to change:    0 0 1 0 1 1 0 ...
Self-enhancement:     1 0 0 0 1 0 1 ...
Conversation:         0 0 0 1 1 0 1 ...
Self-transcendence:   1 1 0 1 0 1 0 ...
```

You compute per-category binary F1-score.

```
Openess to change F1:    0.35
Self-enhancement F1:     0.55
Conversation F1:         0.80
Self-transcendence F1:   0.21
```

You then average per-category scores.
```
Average F1: ~0.48
```

In [ ]:
def model_metric(model, test_dataloader):
    with torch.no_grad():
        all_outputs = []
        all_targets = []

        for premises, conclusions, stances, labels in train_dataloader:
            outputs = model(premises, conclusions, stances)

            all_outputs.append(outputs)
            all_labels.append(labels)

        all_outputs = torch.cat(all_outputs, axis=0)
        all_labels = torch.cat(all_labels, axis=0)

        # all_outputs has shape [N, 4]
        # all_labels has shape [N, 4]

        for i, value_name in enumerate(TOUCHE_VALUES):
            f1_score =

# [Task 4 - 1.0 points] Training and Evaluation

You are now tasked to train and evaluate **all** defined models.

### Instructions

* Train **all** models on the train set.
* Evaluate **all** models on the validation set.
* Pick **at least** three seeds for robust estimation.
* Compute metrics on the validation set.
* Report **per-category** and **macro** F1-score for comparison.

# [Task 5 - 1.0 points] Error Analysis

You are tasked to discuss your results.

### Instructions

* **Compare** classification performance of BERT-based models with respect to baselines.
* Discuss **difference in prediction** between the best performing BERT-based model and its variants.

### Notes

You can check the [original paper](https://aclanthology.org/2022.acl-long.306/) for suggestions on how to perform comparisons (e.g., plots, tables, etc...).

# [Task 6 - 1.0 points] Report

Wrap up your experiment in a short report (up to 2 pages).

### Instructions

* Use the NLP course report template.
* Summarize each task in the report following the provided template.

### Recommendations

The report is not a copy-paste of graphs, tables, and command outputs.

* Summarize classification performance in Table format.
* **Do not** report command outputs or screenshots.
* Report learning curves in Figure format.
* The error analysis section should summarize your findings.

# Submission

* **Submit** your report in PDF format.
* **Submit** your python notebook.
* Make sure your notebook is **well organized**, with no temporary code, commented sections, tests, etc...
* You can upload **model weights** in a cloud repository and report the link in the report.

# FAQ

Please check this frequently asked questions before contacting us

### Model card

You are **free** to choose the BERT-base model card you like from huggingface.

### Model architecture

You **should not** change the architecture of a model (i.e., its layers).

However, you are **free** to play with their hyper-parameters.

### Model Training

You are **free** to choose training hyper-parameters for BERT-based models (e.g., number of epochs, etc...).

### Neural Libraries

You are **free** to use any library of your choice to address the assignment (e.g., Keras, Tensorflow, PyTorch, JAX, etc...)

### Error Analysis

Some topics for discussion include:
   * Model performance on most/less frequent classes.
   * Precision/Recall curves.
   * Confusion matrices.
   * Specific misclassified samples.

# The End